In [1]:
!git clone https://github.com/derInformatiker/AIcrowd-AIBlitz7-Solution.git
!pip install -r AIcrowd-AIBlitz7-Solution/challenge1/requirements.txt
!pip install aicrowd-cli==0.1

fatal: destination path 'AIcrowd-AIBlitz7-Solution' already exists and is not an empty directory.


In [2]:
#RESTART RUNTIME TO USE NEW PACKAGES

In [3]:
API_KEY = ""  # Please enter your API Key from [https://www.aicrowd.com/participants/me]
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [4]:
!aicrowd dataset download --challenge f1-team-classification -j 3

!rm -rf data
!mkdir data

!unzip -q train.zip  -d data/train
!unzip -q val.zip -d data/val
!unzip -q test.zip  -d data/test

!mv train.csv data/train.csv
!mv val.csv data/val.csv
!mv sample_submission.csv data/sample_submission.csv

sample_submission.csv:   0% 0.00/134k [00:00<?, ?B/s]
train.csv:   0% 0.00/569k [00:00<?, ?B/s]
sample_submission.csv: 100% 134k/134k [00:00<00:00, 1.83MB/s]
train.csv: 100% 569k/569k [00:00<00:00, 4.09MB/s]
train.zip:   0% 0.00/391M [00:00<?, ?B/s]
val.csv: 100% 52.9k/52.9k [00:00<00:00, 1.41MB/s]

train.zip:   9% 33.6M/391M [00:00<00:05, 60.7MB/s]

test.zip:   0% 0.00/97.7M [00:00<?, ?B/s]
val.zip:  86% 33.6M/38.9M [00:00<00:00, 60.9MB/s]

val.zip: 100% 38.9M/38.9M [00:00<00:00, 62.9MB/s]
train.zip: 100% 391M/391M [00:05<00:00, 81.6MB/s]
train.zip: 100% 391M/391M [00:05<00:00, 76.0MB/s]
test.zip: 100% 97.7M/97.7M [00:11<00:00, 8.68MB/s]


In [5]:
import shutil

shutil.copy('AIcrowd-AIBlitz7-Solution/challenge1/model.py','model.py')
shutil.copy('AIcrowd-AIBlitz7-Solution/challenge1/dataset.py','dataset.py')

'dataset.py'

In [6]:
 import model
import dataset
import pandas as pd

import albumentations as A
from albumentations.augmentations.transforms import Flip

import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [7]:
if __name__ == '__main__':
    trainer = Trainer(max_epochs = 1, gpus = 1, precision=16, amp_level='O1',deterministic=True)
    
    train_tr = A.Compose([
        A.CenterCrop(256,256,always_apply=True),
        Flip()
    ])
    
    val_tr = A.Compose([
        A.CenterCrop(256,256,always_apply=True)
    ])
    
    model = model.Classifier({'lr':3e-4,'batch_size':64,'train_tr':train_tr,'val_tr':val_tr})
    
    trainer.fit(model)
    trainer.test(model)
    out = trainer.predict(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


Loaded pretrained weights for efficientnet-b3



  | Name   | Type         | Params
----------------------------------------
0 | resnet | EfficientNet | 10.7 M
----------------------------------------
10.7 M    Trainable params
0         Non-trainable params
10.7 M    Total params
42.797    Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


/content/model.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(x)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:51: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1': 1.0, 'test_loss': 9.103668162424583e-06}
--------------------------------------------------------------------------------


In [12]:
import pandas as pd
def writeSub(p):
    labelmap = {0:'redbull',1:'mercedes'}
    test_df = pd.read_csv('data/sample_submission.csv')
    output_list = p.int().tolist()
    output_list = [labelmap[i] for i in output_list]
    test_df['label'] = output_list
    test_df.to_csv(path_or_buf='data/submission.csv',index = False)

In [13]:
output = torch.tensor([])

for i in range(len(out)):
    output = torch.cat((output,torch.tensor(out[i][1]).argmax(1)))

In [14]:
writeSub(output)

In [15]:
!aicrowd submission create -c f1-team-classification -f data/submission.csv

submission.csv ━━━━━━━━━━━━━━━━━━ 100.0% • 135.4/133.8 KB • 425.9 kB/s • 0:00:00
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /usr/local/bin/aicrowd:8 in <module>                                         │
│                                                                              │
│   5 from aicrowd.cli import cli                                              │
│   6 if __name__ == '__main__':                                               │
│   7 │   sys.argv[0] = re.sub(r'(-script\.pyw|\.exe)?$', '', sys.argv[0])     │
│ ❱ 8 │   sys.exit(cli())                                                      │
│                                                                              │
│ /usr/local/lib/python3.7/dist-packages/click/core.py:829 in __call__         │
│                                                                              │
│    826 │                                                                     │
│    827 │   def __call__(se